In [6]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

from pandas_datareader import data as web

## Group Assignment
### Team Number: 9
### Team Member Names: Germain, Padtmesh, Jin
### Team Strategy Chosen: Safe

In [ ]:
def filterData():
    ## Constants
    min_volume = 200000
    min_trading = 20
    min_stockn = 12
    max_stockn= 25


    n=0
    min_weight = 100/(2n)
    max_weight = 25 
    init_inv = 500000


In [ ]:
# efficient frontier calc here
def getOptimalPort():


In [ ]:
# After efficient frontier calc
def createPortFinal():


In [ ]:
def createStocksFinal():


In [39]:
def getHist(tickers):
    # Set defaults for timeframe
    start_date = '2022-01-01'
    end_date = '2022-11-01' # adjustment for non-inclusive param end

    # Create dataframe to hold historic data from tickers
    df_sv = pd.DataFrame()

    # Get the closing price of each ticker and store it as a column in df_sv
    for tick in tickers['Tickers']:
        # Call API for ticker data
        temp = yf.Ticker(tick)

        # Get monthly historical data
        temp_hist = temp.history(start=start_date, end=end_date, interval='1d')

        # Get Closing Price as a DataFrame
        df_sv[tick] = pd.DataFrame(temp_hist['Close'])

    # Remove indeterminate or faulty values
    df_sv.dropna(axis=1,how='all',inplace = True)
    df_sv.dropna(inplace = True)
    return df_sv

In [40]:
# Read in csv file containing tickers
tickers = pd.read_csv('Tickers.csv', names = ['Tickers'])
tickers.head()

,Tickers
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [ ]:
# Get Historic Data for each ticker
hist = getHist(tickers)
hist.head()

# Filter data



In [42]:
# Construct dataframe of percentage change of each stock
hist_pct = hist.pct_change()
hist_pct.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SO,SPG,T,TD.TO,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05 00:00:00-05:00,-0.026600,0.005253,-0.004493,-0.017611,-0.013729,-0.018893,-0.010656,-0.002621,-0.016879,-0.010177,...,0.001174,-0.011965,0.022231,0.006039,-0.022676,-0.020860,-0.002465,0.003015,-0.012564,-0.005275
2022-01-06 00:00:00-05:00,-0.016693,-0.004710,-0.000148,-0.048290,0.017185,-0.006711,0.009164,-0.008119,0.020136,-0.008233,...,-0.001172,0.013227,-0.003815,0.006103,0.010880,-0.000321,-0.040924,0.002769,0.009694,0.027198
2022-01-07 00:00:00-05:00,0.000988,-0.002588,0.003108,-0.019180,0.026187,-0.004288,0.008560,0.019684,0.021816,-0.019806,...,0.006895,-0.011584,0.034163,0.004716,-0.002464,-0.039195,-0.023528,0.005089,0.007016,0.015820
2022-01-10 00:00:00-05:00,0.000116,0.011195,-0.002213,0.006069,0.011031,-0.006570,-0.003670,-0.028724,-0.005083,0.011651,...,-0.001602,-0.002418,0.006466,-0.004094,-0.014559,0.019617,0.013955,-0.017662,-0.029474,0.005902


In [10]:
# Random simulation
# alternative could be to iterate completely over the searchspace
# however assuming an upperbound of 1000(even 100) nCr with 25 stocks would be too slow
# instead we could generate random portfolios up to 10^x iterations
# depending on x we can approximate it to a decent degree of accuracy

def getReturns(pct, weights):
    return (np.sum(pct.mean() * weights)) * 253

# May have to pivot to dot function if @ operator isnt supported
def getStd(pct, weights):
    return np.sqrt(np.transpose(weights) @ (pct.cov() * 253) @ weights)

def randSimulation(pct):
    n = 10**4
    ticker_count = len(pct.columns)
    print(ticker_count)
    list_portfolio_returns = []
    list_portfolio_sd = []
    # Generate n random portfolios
    for p in range(n):
        # Get random values between 0 and 1
        weights = np.random.random(size = ticker_count) 
        # Divide random values by sum to get weights
        weights /= np.sum(weights) 

        list_portfolio_returns.append(getReturns(pct,weights))
        list_portfolio_sd.append(getStd(pct,weights))
        # Convert list to numpy arrays
        port_returns = np.array(object = list_portfolio_returns)
        port_sd = np.array(object = list_portfolio_sd)
    # Max expected return
    print(round(max(port_returns), 4))
    # Min expected return
    print(round(min(port_returns), 4))
    # Max sd
    print(round(max(port_sd), 4))
    # Min sd
    print(round(min(port_sd), 4))


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.